In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import community as community_louvain  
import ast
import random
import tarfile
import json

# Carica dati
edges = pd.read_csv('dataset/spoti/edges.csv')
nodes = pd.read_csv('dataset/spoti/nodes.csv')
nodes_unique = nodes.drop_duplicates(subset=['spotify_id'], keep='first')


In [ ]:
# ============================================================================
# ESTRAZIONE MUSICBRAINZ E MAPPING NAZIONALITÀ
# ============================================================================

def add_nationality_to_nodes(nodes_df):
    """
    Aggiunge la colonna 'nationality' al DataFrame degli artisti usando MusicBrainz.
    """
    
    # DEBUG: Verifica input
    print(f"🔍 DEBUG - Input nodes_df type: {type(nodes_df)}")
    print(f"🔍 DEBUG - Input nodes_df is None: {nodes_df is None}")
    
    if nodes_df is None:
        print("❌ ERRORE: nodes_df è None all'ingresso della funzione!")
        return None
    
    print(f"🔍 DEBUG - nodes_df shape: {nodes_df.shape}")
    print(f"🔍 DEBUG - nodes_df columns: {list(nodes_df.columns)[:5]}...")
    
    print("\nCaricamento artisti da MusicBrainz...")
    artists_dict = {}
    
    # Leggi direttamente dal file artist
    try:
        with open('dataset/spoti/artist/mbdump/artist', 'r', encoding='utf-8') as f:
            count = 0
            matched = 0
            for line in f:
                try:
                    artist = json.loads(line)
                    name = artist.get('name', '').lower().strip()
                    
                    # Prendi il nome del paese dall'oggetto area
                    area = artist.get('area')
                    if area and isinstance(area, dict):
                        # Verifica se è un Country guardando i codici ISO
                        iso_codes = area.get('iso-3166-1-codes', [])
                        if iso_codes:  # Se ha codice ISO paese, è un Country
                            country_name = area.get('name')
                            if name and country_name:
                                if name not in artists_dict:
                                    artists_dict[name] = country_name
                                    matched += 1
                    
                    count += 1
                    if count % 100000 == 0:
                        print(f"  Processati {count} artisti... (matchati: {matched})")
                        
                except Exception as e:
                    continue
        
        print(f"Artisti totali processati: {count}")
        print(f"Artisti con nazionalità estratti: {len(artists_dict)}")
        
    except FileNotFoundError as e:
        print(f"❌ ERRORE: File non trovato - {e}")
        return None
    except Exception as e:
        print(f"❌ ERRORE durante lettura file: {e}")
        return None
    
    # DEBUG: Verifica dizionario
    print(f"\n🔍 DEBUG - artists_dict size: {len(artists_dict)}")
    if len(artists_dict) > 0:
        print(f"🔍 DEBUG - Primi 3 artisti del dizionario:")
        for i, (name, country) in enumerate(list(artists_dict.items())[:3]):
            print(f"    {name} -> {country}")
    
    # Aggiungi nationality al DataFrame
    print(f"\n🔍 DEBUG - Prima di copy(), nodes_df type: {type(nodes_df)}")
    
    try:
        nodes_df = nodes_df.copy()
        print(f"🔍 DEBUG - Dopo copy(), nodes_df shape: {nodes_df.shape}")
        
        # Verifica che esista la colonna 'name'
        if 'name' not in nodes_df.columns:
            print(f"❌ ERRORE: Colonna 'name' non trovata!")
            print(f"   Colonne disponibili: {list(nodes_df.columns)}")
            return None
        
        print(f"🔍 DEBUG - Colonna 'name' trovata, primi 3 valori:")
        print(f"    {nodes_df['name'].head(3).tolist()}")
        
        nodes_df['name_lower'] = nodes_df['name'].str.lower().str.strip()
        print(f"🔍 DEBUG - Creata colonna name_lower, primi 3 valori:")
        print(f"    {nodes_df['name_lower'].head(3).tolist()}")
        
        nodes_df['nationality'] = nodes_df['name_lower'].map(artists_dict)
        print(f"🔍 DEBUG - Creata colonna nationality")
        
        nodes_df.drop(columns=['name_lower'], inplace=True)
        print(f"🔍 DEBUG - Rimossa colonna name_lower")
        
        matched_nodes = nodes_df['nationality'].notna().sum()
        total = len(nodes_df)
        
        print(f"\n{'='*50}")
        print(f"RISULTATI MATCHING")
        print(f"{'='*50}")
        print(f"Artisti matchati: {matched_nodes}/{total} ({100*matched_nodes/total:.1f}%)")
        print(f"Artisti senza nazionalità: {total - matched_nodes}")
        
        print(f"\n🔍 DEBUG - Prima di return, nodes_df type: {type(nodes_df)}")
        print(f"🔍 DEBUG - Prima di return, nodes_df is None: {nodes_df is None}")
        
        return nodes_df
        
    except Exception as e:
        print(f"❌ ERRORE durante processing del DataFrame: {e}")
        import traceback
        traceback.print_exc()
        return None


# DEBUG: Verifica nodes_unique prima della chiamata
print("="*70)
print("VERIFICA PRIMA DELLA CHIAMATA")
print("="*70)
print(f"🔍 nodes_unique type: {type(nodes_unique)}")
print(f"🔍 nodes_unique is None: {nodes_unique is None}")
if nodes_unique is not None:
    print(f"🔍 nodes_unique shape: {nodes_unique.shape}")
    print(f"🔍 nodes_unique columns: {list(nodes_unique.columns)[:5]}...")
print()

# Esegui il mapping
result = add_nationality_to_nodes(nodes_unique)

# DEBUG: Verifica risultato
print("\n" + "="*70)
print("VERIFICA DOPO LA CHIAMATA")
print("="*70)
print(f"🔍 result type: {type(result)}")
print(f"🔍 result is None: {result is None}")
if result is not None:
    print(f"🔍 result shape: {result.shape}")
    print(f"🔍 'nationality' in columns: {'nationality' in result.columns}")
    nodes_unique = result
else:
    print("❌ La funzione ha restituito None!")


In [ ]:
# ============================================================================
# REFINE NATIONALITY USANDO I GENERI
# ============================================================================

def refine_nationality_with_genres(nodes_df):
    """
    Affina la nazionalità degli artisti dando priorità ai generi musicali
    che contengono riferimenti geografici espliciti.
    
    Se un artista ha generi con keyword nazionali, sovrascrive la nationality 
    di MusicBrainz (utile per artisti con nationality mancante o ambigua).
    
    Parametri:
    - nodes_df: DataFrame con colonne 'genres' e 'nationality'
    
    Returns:
    - DataFrame modificato con nationality aggiornata
    """
    
    # Mapping generi -> nazionalità (espandibile)
    GENRE_TO_COUNTRY = {
        # Italia
        "italian": "Italy",
        "ital": "Italy",
        
        # Francia
        "french": "France",
        "fran": "France",
        
        # Germania
        "german": "Germany",
        "deutsch": "Germany",
        
        # Spagna
        "spanish": "Spain",
        
        # UK
        "british": "United Kingdom",
        "uk": "United Kingdom",
        "english": "United Kingdom",
        
        # USA
        "american": "United States",
        
        # Nordici
        "swedish": "Sweden",
        "norwegian": "Norway",
        "danish": "Denmark",
        "finnish": "Finland",
        "icelandic": "Iceland",
        
        # Altri paesi
        "brazilian": "Brazil",
        "portuguese": "Portugal",
        "mexican": "Mexico",
        "argentinian": "Argentina",
        "japanese": "Japan",
        "korean": "South Korea",
        "chinese": "China",
        "indian": "India",
        "turkish": "Turkey",
        "greek": "Greece",
        "polish": "Poland",
        "russian": "Russia",
        "dutch": "Netherlands",
        "belgian": "Belgium",
        "austrian": "Austria",
        "swiss": "Switzerland",
        "canadian": "Canada",
        "australian": "Australia",
        "irish": "Ireland",
        "scottish": "United Kingdom",  # Scozia -> UK
        "welsh": "United Kingdom",      # Galles -> UK
    }
    
    print("Raffinamento nazionalità tramite analisi dei generi...")
    
    nodes_df = nodes_df.copy()
    refined_count = 0
    added_count = 0
    
    for idx, row in nodes_df.iterrows():
        genres = row['genres']
        current_nationality = row['nationality']
        
        # Salta se genres è vuoto o NaN
        if pd.isna(genres) or genres == '[]' or genres == '':
            continue
        
        # Converti stringa di lista in stringa lowercase per il matching
        genres_lower = str(genres).lower()
        
        # Cerca keyword nei generi
        detected_country = None
        for keyword, country in GENRE_TO_COUNTRY.items():
            if keyword in genres_lower:
                detected_country = country
                break  # Prendi il primo match
        
        # Se troviamo una nazionalità nei generi
        if detected_country:
            # Se la nationality è vuota, aggiungiamola
            if pd.isna(current_nationality):
                nodes_df.at[idx, 'nationality'] = detected_country
                added_count += 1
            # Se è diversa da quella di MusicBrainz, diamo priorità ai generi
            elif current_nationality != detected_country:
                nodes_df.at[idx, 'nationality'] = detected_country
                refined_count += 1
    
    print(f"\n{'='*50}")
    print(f"RISULTATI RAFFINAMENTO")
    print(f"{'='*50}")
    print(f"Nazionalità aggiunte (prima assenti): {added_count}")
    print(f"Nazionalità modificate (da MusicBrainz): {refined_count}")
    print(f"Totale artisti con nazionalità: {nodes_df['nationality'].notna().sum()}")
    
    return nodes_df

In [ ]:
# Debug: controlla il formato dei generi
riria = nodes_unique[nodes_unique['name'] == 'Riria.']
print(f"Generi di Riria: {riria['genres'].values}")
print(f"Tipo di dato: {type(riria['genres'].values[0])}")

# Se esiste la riga, stampane anche il contenuto completo
if len(riria) > 0:
    print(f"\nRiga completa:")
    print(riria)
    print(f"\nNazionalità attuale: {riria['nationality'].values[0]}")
else:
    print("❌ Artista 'Riria.' non trovato")


In [ ]:
# Step 2: Raffina usando i generi
print("\n" + "="*70)
print("STEP 2: RAFFINAMENTO CON GENERI MUSICALI")
print("="*70)
nodes_unique = refine_nationality_with_genres(nodes_unique)

In [ ]:
# Correzione: devi iterare sulle righe del DataFrame
for idx, row in nodes_unique.iterrows():
    if row['name'] == "MACE":
        print(row)


In [ ]:
# Salva nodes_unique come nuovo nodes.csv
nodes_unique.to_csv('dataset/spoti/nodes.csv', index=False)
print("✓ File salvato: dataset/spoti/nodes.csv")
print(f"  Shape: {nodes_unique.shape}")
print(f"  Colonne: {list(nodes_unique.columns)}")
print(f"  Artisti con nazionalità: {nodes_unique['nationality'].notna().sum()}")


In [21]:
import pandas as pd
import ast
import sys

# Carica il CSV
df = pd.read_csv('dataset/spoti/nodes.csv', low_memory=False)

# Funzione per estrarre generi dalla colonna 'genres'
def extract_genres(genre_str):
    try:
        if pd.isna(genre_str) or genre_str == '[]' or genre_str == '':
            return []
        if isinstance(genre_str, str):
            return ast.literal_eval(genre_str)
        return []
    except:
        return []

# Estrai tutti i generi
all_genres = []
for genres in df['genres']:
    all_genres.extend(extract_genres(genres))

# Rimuovi duplicati e ordina
unique_genres = sorted(set(all_genres))

with open('dataset/spoti/all_genres.txt', 'w', encoding='utf-8') as f:
    f.write("[\n")
    for genre in unique_genres:
        f.write(f"    '{genre}',\n")
    f.write("]\n")

print(f"Generi salvati in 'dataset/spoti/all_genres.txt'")
print(f"Numero totale di generi unici: {len(unique_genres)}")

Generi salvati in 'dataset/spoti/all_genres.txt'
Numero totale di generi unici: 4853


In [22]:
import pandas as pd

with open('dataset/spoti/all_genres.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Estrai i generi
genres = [line.strip().strip("',[]") for line in lines if line.strip() and not line.startswith('Numero') and not line.startswith('Array')]

# Regole base di mappatura
def map_genre(genre):
    genre_lower = genre.lower()
    
    if 'pop' in genre_lower or 'pop' == genre_lower:
        return 'Pop'
    elif 'rock' in genre_lower:
        return 'Rock'
    elif any(word in genre_lower for word in ['hip hop', 'rap', 'trap', 'drill', 'boom bap', 'grime']):
        return 'Hip Hop / Rap'
    elif any(word in genre_lower for word in ['electronic', 'techno', 'house', 'trance', 'ambient', 'dubstep', 'dnb', 'drum and bass', 'edm', 'breakbeat', 'hardstyle', 'electro', 'idm', 'chiptune', 'vaporwave', 'lo-fi', 'synth']):
        return 'Elettronica / Dance'
    elif 'jazz' in genre_lower:
        return 'Jazz'
    elif any(word in genre_lower for word in ['classical', 'orchestra', 'choir', 'baroque', 'symphony', 'concerto', 'chamber', 'opera', 'piano solo', 'violin solo']):
        return 'Classica / Orchestrale'
    elif any(word in genre_lower for word in ['folk', 'traditional', 'acoustic', 'singer-songwriter', 'americana', 'bluegrass']):
        return 'Folk / Tradizionale'
    elif any(word in genre_lower for word in ['country', 'americana', 'honky tonk', 'western']):
        return 'Country / Americana'
    elif 'blues' in genre_lower:
        return 'Blues'
    elif 'metal' in genre_lower:
        return 'Metal'
    elif any(word in genre_lower for word in ['punk', 'hardcore', 'emo', 'screamo', 'post-hardcore']):
        return 'Punk / Hardcore'
    elif any(word in genre_lower for word in ['funk', 'disco', 'boogie']):
        return 'Funk / Disco'
    elif any(word in genre_lower for word in ['r&b', 'soul', 'motown', 'neo-soul']):
        return 'R&B / Soul'
    elif any(word in genre_lower for word in ['reggae', 'dancehall', 'ska', 'rocksteady', 'dub']):
        return 'Reggae / Dancehall'
    elif any(word in genre_lower for word in ['latin', 'salsa', 'bachata', 'cumbia', 'merengue', 'reggaeton', 'tango', 'samba', 'bossanova', 'forro']):
        return 'Latina / Caraibica'
    elif any(word in genre_lower for word in ['gospel', 'christian', 'worship', 'religious', 'spiritual', 'ccm']):
        return 'Religiosa / Spirituale'
    elif any(word in genre_lower for word in ['soundtrack', 'score', 'theme', 'ost', 'film music', 'video game music']):
        return 'Soundtrack / Colonne sonore'
    elif any(word in genre_lower for word in ['comedy', 'parody', 'humor', 'satire']):
        return 'Commedia / Satira'
    elif any(word in genre_lower for word in ['world', 'ethnic', 'african', 'arab', 'indian', 'celtic', 'flamenco', 'klezmer', 'oriental']):
        return 'World / Etnica'
    elif any(word in genre_lower for word in ['indie', 'Indie']):
        return 'Indie'
    else:
        return 'Altri / Specifici'

mapped = [(genre, map_genre(genre)) for genre in genres]
df = pd.DataFrame(mapped, columns=['genere', 'macro_categoria'])

# Salva in CSV
df.to_csv('dataset/spoti/all_genres_mapped.csv', index=False, encoding='utf-8')

print(f"Mappati {len(df)} generi.")
print(df['macro_categoria'].value_counts())

Mappati 4855 generi.
macro_categoria
Altri / Specifici              1498
Hip Hop / Rap                   523
Pop                             498
Elettronica / Dance             367
Indie                           341
Rock                            299
Folk / Tradizionale             232
Classica / Orchestrale          192
Punk / Hardcore                 163
Metal                           150
Jazz                            122
Reggae / Dancehall               94
Latina / Caraibica               66
Funk / Disco                     55
R&B / Soul                       54
Religiosa / Spirituale           51
Blues                            38
Country / Americana              36
Soundtrack / Colonne sonore      36
World / Etnica                   31
Commedia / Satira                 9
Name: count, dtype: int64


In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset/spoti/all_genres_mapped.csv')

# Estraggo tutti le righe con: "Altri / Specifici"
altri_generi = df[df['macro_categoria'] == 'Altri / Specifici']

num_altri = len(altri_generi)
print(f"Numero di generi nella categoria 'Altri / Specifici': {num_altri}")

nomi_generi = []
for genere in altri_generi['genere']:
    if pd.isna(genere):
        nomi_generi.append('')
    elif isinstance(genere, float):
        nomi_generi.append(str(int(genere)) if genere.is_integer() else str(genere))
    else:
        nomi_generi.append(str(genere))

with open('dataset/spoti/other_genres.txt', 'w', encoding='utf-8') as f:
    for genere in nomi_generi:
        if genere.strip():  # Salta righe vuote
            f.write(genere + '\n')

print(f"Lista salvata in 'dataset/spoti/other_genres.txt'")
for i, genere in enumerate(nomi_generi[:20]):
    if genere.strip():
        print(f"{i+1}. {genere}")

validi = sum(1 for g in nomi_generi if g.strip())
print(f"\nGeneri validi (non vuoti): {validi}")

Numero di generi nella categoria 'Altri / Specifici': 1498
Lista salvata in 'dataset/spoti/other_genres.txt'
2. 432hz
3. 48g
4. 8-bit
5. 8d
6. a cappella
7. a3
8. abstract
9. abstract beats
10. abstractro
11. accordion
12. acousmatic
13. adoracao
14. adoracion
15. adult standards
16. adventista
17. afrikaans
18. afro-cuban percussion
19. afrobeat
20. afrobeat brasileiro

Generi validi (non vuoti): 1496


In [24]:
import pandas as pd
import re

df = pd.read_csv('dataset/spoti/all_genres_mapped.csv')

# Leggi il file txt con la mappatura eseguita con l'AI separatamente
with open('dataset/spoti/ai_mapped_genres.txt', 'r', encoding='utf-8') as f:
    contenuto = f.read()

mappatura_generi = {}
macro_categoria_corrente = None

for linea in contenuto.split('\n'):
    linea = linea.strip()
    
    if linea.startswith('## **') and linea.endswith('**'):
        macro_categoria_corrente = linea.replace('## **', '').replace('**', '').strip()
    elif linea and not linea.startswith('#') and macro_categoria_corrente:
        mappatura_generi[linea.lower()] = macro_categoria_corrente

# Aggiorna il dataframe: per ogni riga con macro_categoria "Altri / Specifici", cerca il genere corrispondente nel dizionario di mappatura
df.loc[df['macro_categoria'] == 'Altri / Specifici', 'macro_categoria'] = \
    df.loc[df['macro_categoria'] == 'Altri / Specifici', 'genere'].str.lower().map(mappatura_generi)

df.to_csv('dataset/spoti/all_genres_mapped_v2.csv', index=False)

print("File generato con successo: dataset/spoti/all_genres_mapped_v2.csv")
print(f"Totale generi mappati: {len(mappatura_generi)}")

File generato con successo: dataset/spoti/all_genres_mapped_v2.csv
Totale generi mappati: 1492


In [ ]:

import pandas as pd
import ast

nodes_df = pd.read_csv('dataset/spoti/nodes.csv', low_memory=False)
generi_df = pd.read_csv('dataset/spoti/all_genres_mapped_v2.csv')

mappatura = dict(zip(generi_df['genere'].str.lower(), generi_df['macro_categoria']))

def ottieni_macro_genres(generi_str, row_index):
    if pd.isna(generi_str):
        return '[]'
    
    try:
        generi_list = ast.literal_eval(generi_str)
        
        macro_genres = []
        generi_non_trovati = []
        
        for genere in generi_list:
            genere_lower = genere.lower().strip()
            if genere_lower in mappatura:
                macro_cat = mappatura[genere_lower]
                if macro_cat not in macro_genres:
                    macro_genres.append(macro_cat)
            else:
                generi_non_trovati.append(genere)
        
        if generi_non_trovati:
            print(f"Riga {row_index}: Generi non mappati: {generi_non_trovati}")
        
        return str(macro_genres)
    except Exception as e:
        print(f"Riga {row_index}: Errore nel parsing: {e}")
        return '[]'

nodes_df['macro_genres'] = nodes_df.apply(lambda row: ottieni_macro_genres(row['genres'], row.name), axis=1)
nodes_df.to_csv('dataset/spoti/nodes2.csv', index=False)
#nodes_df.to_csv('dataset/spoti/nodes.csv', index=False)

print("File generato con successo: 'dataset/spoti/nodes2.csv'")
print(f"Totale righe processate: {len(nodes_df)}")

File generato con successo: nodes2.csv
Totale righe processate: 156320


In [34]:
import pandas as pd
import ast

nodes_df = pd.read_csv('dataset/spoti/nodes2.csv', low_memory=False)

def is_empty_array(value):
    if pd.isna(value):
        return True
    
    try:
        parsed = ast.literal_eval(str(value))
        return len(parsed) == 0
    except:
        return False

# Conta gli elementi con genres vuoti
genres_vuoti = nodes_df['genres'].apply(is_empty_array).sum()

# Conta gli elementi con macro_genres vuoti
macro_genres_vuoti = nodes_df['macro_genres'].apply(is_empty_array).sum()

# Conta gli elementi che hanno entrambi vuoti
entrambi_vuoti = (nodes_df['genres'].apply(is_empty_array) & 
                  nodes_df['macro_genres'].apply(is_empty_array)).sum()

# Conta gli elementi con genres non vuoto ma macro_genres vuoto
genres_pieni_macro_vuoti = (~nodes_df['genres'].apply(is_empty_array) & 
                             nodes_df['macro_genres'].apply(is_empty_array)).sum()

# Statistiche
totale_righe = len(nodes_df)

print("=" * 50)
print("STATISTICHE GENRES VUOTI")
print("=" * 50)
print(f"Totale righe: {totale_righe}")
print(f"\nGenres vuoti []: {genres_vuoti} ({genres_vuoti/totale_righe*100:.2f}%)")
print(f"Macro_genres vuoti []: {macro_genres_vuoti} ({macro_genres_vuoti/totale_righe*100:.2f}%)")
print(f"Entrambi vuoti []: {entrambi_vuoti} ({entrambi_vuoti/totale_righe*100:.2f}%)")
print(f"\nGenres NON vuoto ma Macro_genres vuoto []: {genres_pieni_macro_vuoti} ({genres_pieni_macro_vuoti/totale_righe*100:.2f}%)")
print("=" * 50)

STATISTICHE GENRES VUOTI
Totale righe: 156320

Genres vuoti []: 102121 (65.33%)
Macro_genres vuoti []: 102121 (65.33%)
Entrambi vuoti []: 102121 (65.33%)

Genres NON vuoto ma Macro_genres vuoto []: 0 (0.00%)


In [38]:
import os

# Lista dei file d'appoggio da cancellare
files_to_delete = [
    'dataset/spoti/all_genres.txt',
    'dataset/spoti/all_genres_mapped.csv',
    'dataset/spoti/other_genres.txt',
    'dataset/spoti/all_genres_mapped_v2.csv'
]

# Cancella i file
for file_path in files_to_delete:
    try:
        if os.path.exists(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f" Errore nella cancellazione di {file_path}: {e}\n")

print("Operazione completata.")

Operazione completata.
